# Polymer generation in ligpargen

I used `O=C(O)c1cccc(C(=O)CCO)c1`, a smiles string of the PET monomer in ligpargen to create the opls-AA representation of the PET monomer. I then corrected the opls_aa type assignments. Note for each generated molecule, the structure will be different. Thus care needs to be taken when generalizing these results for all PET's generated in ligpargen. The results and FF modifications only work for this molecule

Default setup has a net charge of -0.0001. I modified the species with the most negative charge by adding 0.0001 to the charge on that species to get it neutral. This turned out to be the terminal H terminus oxygen which I changed from -0.5556 to -0.5555. To test out how this works, I then ran a simulation of PET monomer in water.

I did only the energy minimization step with spc water with 3 PET monomers in a 5nm box. Presented below is a snapshot at the final timestep

![PET monomer in 5nm box](test_PET_monomer_ligpargen.png)

Next thing I did was I added the PET monomer specifications to the opls_aa forcefield. This was done by following the instructions [here](https://manual.gromacs.org/current/how-to/topology.html). 

First, I modified the `aminoacids.rtp` file with the appropriate atom identities and bonds. I then modified the `ffbonded.itp` to include the dihedrals of the benzene ring labelled `improper_CAR_HAR`. The ester linkage is already present and labelled `define improper_O_C_X_Y`.

I then flattened the PET monomer to 2D to allow for easy translation of the monomer. I call this file `2D_PET_final.gro`

![Single PET monomer in 2D after flattening](2D_PET.png)

Next, the following command was run: `gmx pdb2gmx -f PET.gro` to convert the `.gro` file to a topology file with format `.top`. The converted PET monomer is shown below to demonstrate that the conversion does indeed work

![Single PET monomer in 2D after flattening](pdb2gmx_test.png)

I had to manually delete some of the bonds that VMD thought was there but the structure is reproduced succesfully. I then tried to run the `pdb2gmx` generated topology file with gromacs which failed during the energy minimization step. This was due to some of the angles and dihedrals not having default values. This was solved by adding the default values using the atomnames from `ffnonbonded.itp` into `ffbonded.itp`. I added these by looking at the line that was causing an issue in `topology.top` and adding the corresponding bond names and default values from `PET.itp` obtained from ligpargen to the appropriate columns in `ffnonbonded.itp`. After conversion, the topol.top still says other instead of PET. 

I then created the `make_polymer.py` file which takes the exact `PET.gro` file in this subfolder and extends the chain to however many monomeric units desired.

In [21]:
!python3 make_polymer.py 3

Test case with DP of 3 is shown below with the unnecessary bonds deleted

![PET trimer](DP_3_PET.png)

When trying to use pdb2gmx on this file, an error appears which requires hydrogens to be added to the `aminoacids.hdb` file to allow simulation of the polymer